# How to control the verbosity of sos output and the number of running jobs

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * `-v` controls logging level from `error`, `warning`, `info` to `debug`
  * `-v1` displays all workflow output, `-v2` displays workflow output plus workflow information, `-v3` displays some debug information.
  * `-v0` suppress all user output (except for errors) and introduces a special text-based progress bar
  * Environment variable `DBG_DEBUG` can be used to generate topic-specific debug information
  * `-j` controls number of concurrent processes

## Logging level

SoS uses a logging system to output all sorts of information during the execution of workflows. The amount of output can be controlled by logging level, which can be `error` (0), `warning` (1), `info` (2), and `debug` (3). The default logging level for SoS is `info`.

For example, logging at `info` level would produce message indicating the steps executed and input output files, while `-v0` will suppress all but error messages. It is worth noting that **if you are using SoS Notebook, all logging messages will be sent to the side panel if the console panel is open**.

<div class="bs-callout bs-callout-primary" role="alert">
    <h4>Text-based progress bar with <code>-v0</code></h4>
    <p>If you have a large workflow with lots of steps, the default output can be overwhelming. In this case you can use <code>-v0</code>, which not only suppress all `INFO` messages and script output, but also introduces a text-based progress bar from which you can learn the progress of the workflow easily.</p>
</div>

In [1]:
!touch a.txt

%run -v0
[10]
print(f'{step_name}')
[20]
input: 'a.txt'
print(f'{step_name}')
[30]
print(f'{step_name}')      
[40]
print(f'{step_name}')      

[####] 4 steps processed (4 jobs completed)


In `-v1` mode, all output from user statements will be outputted

In [2]:
!touch a.txt

%run -v1
[10]
print(f'{step_name}')
[20]
input: 'a.txt'
print(f'{step_name}')
[30]
print(f'{step_name}')      
[40]
print(f'{step_name}')      

10
20
30
40


SoS will output additional logging information at level `-v2`

In [3]:
!touch a.txt

%run -v2
[10]
print(f'{step_name}')
[20]
input: 'a.txt'
print(f'{step_name}')
[30]
print(f'{step_name}')      
[40]
print(f'{step_name}')      

INFO: Running 10 :

10


INFO: Running 20 :

20


INFO: Running 30 :

30


INFO: Running 40 :

40


INFO: Workflow default (ID=fe2086f46821e193) is executed successfully with 4 completed steps.

<div class="bs-callout bs-callout-primary" role="alert">
    <h4>Logging messages are by default sent to console panel in SoS Notebook</h4>
    <p>If you are executing the workflow in SoS notebook with the console panel open, all logging messages will be sent to the console panel. In particular, the output of your workflow in `-v2` mode in the notebook will be identical to `-v1`</p>
</div>

## `SOS_DEBUG` and `~/sos_debug.log` 

This option is designed for developers but if you would like to peak into what SoS is actually doing, you can set an environment variable `SOS_DEBUG`, with `,` separated values of

* `ALL`: all debug messages
* Topics such as `CONTROLLER`, `TASK`, `WORKER`, `EXECUTOR` to keep only debug information of specific topics. The actual values can vary so you should check the output of command `sos run -h` for acceptable topics.
* A filename with an extension, or `-` for standard error output, or nothing to use `~/sos_debug.log` as default.

For example,
```
export SOS_DEBUG=ALL
```
will save a large amount of debug information to `~/sos_debug.log`, regardless of verbosity level set by `-v`,

```
export SOS_DEBUG=TASK,-
```
will send task related debug information to the terminal, and

```
export SOS_DEBUG=ALL,myrun.log
```
will save all debug information in `myrun.log`, which can be useful if you notice something wrong with SoS or your workflow and would like to submit a bug report.

## Control the number of running jobs

<div class="bs-callout bs-callout-primary" role="alert">
    <h4>Control number of workers with option <code>-j</code></h4>
    <p>Option <code>-j</code> sets the number of workers for SoS. The default value is half of the number of processes.</p>
</div>

SoS dispatch steps, substeps, and nested workflows to dedicated workers. The maximum number of concurrent workers can be controlled by the `-j` option. Although the default value, which is half of the number of processes of the computer, is good most of the time, you might want to control the number of workers depending on your running environment (e.g. on a shared server) or on the size of your workflow. Note that `-j` specifies the number of workers so the total number of processes started by `sos` can be at most `n + 1` with option `-j n`.

<div class="bs-callout bs-callout-warning" role="alert">
    <h4>Dedicated workers for blocking nested workflows</h4>
    <p>Ocassionally you might see more than <code>n + 1</code> SoS processes when you execute a workflow with option `-j n`. This is because the workflow consists of blocking nested workflows, namely nested workflows not as the last statement of a step. For example, the nested workflow `A` in the following SoS step has to be executed in a blocking fashion before `some_other_computation` can be done,</p>
    <pre>
        [10]
        sos_run('A')
        some_other_computation()
        </pre>
    <p>So SoS spawns a separate process for nested workflow <code>A</code> so that it will not affect the concurrent execution of other parts of the workflow.</p>    
</div>

## Remote workers

A more advanced version of option `-j` allows the specification of multiple workers on multiple remote workers. For example, if you have `host1` and `host2` directly accessible by the localhost (through public-key authentication) and with SoS installed, you can create workers on these hosts through

```bash
sos run script workflow -j 4 host1:4 host2:4
```
which will start 4 workers locally, 4 workers on `host1` and 4 workers on `host2`.

In practice, however, this option is rarely used directly and is specified on cluster systems when multiple nodes are allocated to execute a workflow.

## Further reading

* [Remote execution of workflows](remote_execution.html)